In [ ]:


import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/root/autodl-tmp/huggingface'
os.environ['MODELSCOPE_CACHE'] = '/root/autodl-tmp/modelscope/hub'

In [ ]:
init = False
if init:
    !pip install huggingface_hub peft opencv-python einops gradio spaces GPUtil
    !conda install -c conda-forge gxx=11 gcc=11 -y
    %cd /root/autodl-tmp/nunchaku
    !git submodule init
    !git submodule update
    !VERBOSE=1 pip install -e . -v
    !cat /root/autodl-tmp/nunchaku/build/temp*/compile_commands.json  # 如果存在

%cd /root/autodl-fs

In [ ]:
from huggingface_hub import hf_hub_download
import os

qmodel_path = "mit-han-lab/svdquant-models/svdq-int4-flux.1-dev.safetensors"

if not os.path.exists(qmodel_path):
    hf_repo_id = os.path.dirname(qmodel_path)
    filename = os.path.basename(qmodel_path)
    qmodel_path = hf_hub_download(repo_id=hf_repo_id, filename=filename)

In [ ]:

import torch

from nunchaku.pipelines import flux as nunchaku_flux

pipeline = nunchaku_flux.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    torch_dtype=torch.bfloat16,
    qmodel_path="mit-han-lab/svdquant-models/svdq-int4-flux.1-dev.safetensors",  # download from Huggingface
    qencoder_path="mit-han-lab/svdquant-models/svdq-w4a16-t5.pt",
).to("cuda")
image = pipeline(
    "A cat holding a sign that says hello world",
    num_inference_steps=28,
    guidance_scale=3.5,
).images[0]


In [ ]:

image